# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
J. Li  ->  J. Li  |  ['J. Li']
S. Li  ->  S. Li  |  ['S. Li']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']


Arxiv has 45 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.13050


extracting tarball to tmp_2310.13050...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.13050/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

/tmp/ipykernel_2152/3009462391.py:49: LatexWarning: 2310.13050 did not run properly
bad escape \e at position 37
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2310.13278...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.13489


extracting tarball to tmp_2310.13489...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.13278-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.13278) | **MaNGA DynPop -- VI. Matter density slopes from dynamical models of 6000  galaxies vs cosmological simulations: the interplay between baryonic and dark  matter**  |
|| <mark>S. Li</mark>, et al. |
|*Appeared on*| *2023-10-23*|
|*Comments*| *17 pages, 14 figures; Submitted to MNRAS on 19 October 2023*|
|**Abstract**| We try to understand the trends in the mass density slopes as a function of galaxy properties. We use the results from the best Jeans Anisotropic Modelling (JAM) of the integral-field stellar kinematics for near 6000 galaxies from the MaNGA DynPop project, with stellar masses of $10^{9-12}\ {\rm M_{\odot}}$, including both early-type and late-type galaxies. We use the mass-weighted density slopes for the stellar $\overline{\gamma}_*$, dark $\overline{\gamma}_{\rm DM}$, and total $\overline{\gamma}_{\rm T}$ mass from Zhu et al. (2023b). The $\overline{\gamma}_{\rm T}$ approaches a constant value of 2.2 for high $\sigma_{\rm e}$ galaxies, and flattens for lg$(\sigma_{\rm e}/{\rm km\ s^{-1}})\lesssim2.3$, reaching 1.5 for lg$(\sigma_{\rm e}/{\rm km\ s^{-1}})\approx1.8$. The total and stellar slopes track each other tightly, with $\overline{\gamma}_{\rm T}\approx\overline{\gamma}_*-0.174$ over the full $\sigma_{\rm e}$ range. This confirms the dominance of stellar matter within $R_{\rm e}$. We also show that there is no perfect conspiracy between baryonic and dark matter, as $\overline{\gamma}_*$ and $\overline{\gamma}_{\rm DM}$ do not vary inversely within the $\sigma_{\rm e}$ range. We find that the central galaxies from TNG50 and TNG100 simulations do not reproduce the observed galaxy mass distribution, which we attribute to the overestimated dark matter fraction, possibly due to a constant IMF and excessive adiabatic contraction effects in the simulations. Finally, we present the stacked dark matter density profiles and show that they are slightly steeper than the pure dark matter simulation prediction of $\overline{\gamma}_{\rm DM}\approx1$, suggesting moderate adiabatic contraction in the central region of galaxies. Our work demonstrate the power of stellar dynamics modelling for probing the interaction between stellar and dark matter and testing galaxy formation theories. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.13489-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.13489) | **Maser Investigation toward Off-Plane Stars (MIOPS): detection of SiO  masers in the Galactic thick disk and halo**  |
|| W. Yang, et al. -- incl., <mark>J. Li</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2023-10-23*|
|*Comments*| *15 pages, 6 figures, 3 tables, accepted for publication in ApJ*|
|**Abstract**| Studying stars that are located off the Galactic plane is important for understanding the formation history of the Milky Way. We searched for SiO masers toward off-plane O-rich asymptotic giant branch (AGB) stars from the catalog presented by Mauron et al. (2019) in order to shed light on the origin of these objects. A total of 102 stars were observed in the SiO $J$=1-0, $v=1$ and 2 transitions with the Effelsberg-100 m and Tianma-65 m telescopes. SiO masers were discovered in eight stars, all first detections. The measured maser velocities allow the first estimates of the host AGB stars' radial velocities. We find that the radial velocities of three stars (namely G068.881-24.615, G070.384-24.886, and G084.453-21.863) significantly deviate from the values expected from Galactic circular motion. The updated distances and 3D motions indicate that G068.881$-$24.615 is likely located in the Galactic halo, while G160.648-08.846 is probably located in the Galactic thin disk, and the other six stars are probably part of the Galactic thick disk. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.13050-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.13050) | **The Three-Phase Evolution of the Milky Way**  |
|| V. Chandra, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>R. Andrae</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2023-10-23*|
|*Comments*| *21 pages, 12 figures, submitted to ApJ*|
|**Abstract**| We illustrate the formation and evolution of the Milky Way over cosmic time, utilizing a sample of 10 million red giant stars with full chemodynamical information, including metallicities and $\alpha$-abundances from low-resolution Gaia XP spectra. The evolution of angular momentum as a function of metallicity - a rough proxy for stellar age, particularly for high-[$\alpha$/Fe] stars - displays three distinct phases: the disordered and chaotic protogalaxy, the kinematically-hot old disk, and the kinematically-cold young disk. The old high-$\alpha$ disk starts at [Fe/H] $\approx -1.0$, 'spinning up' from the nascent protogalaxy, and then exhibits a smooth 'cooldown' toward more ordered and circular orbits at higher metallicities. The young low-$\alpha$ disk is kinematically cold throughout its metallicity range, with its observed properties modulated by a strong radial gradient. We interpret these trends using Milky Way analogs from the TNG50 cosmological simulation, identifying one that closely matches the kinematic evolution of our Galaxy. This halo's protogalaxy spins up into a relatively thin and misaligned high-$\alpha$ disk at early times, which is subsequently heated and torqued by a major gas-rich merger. The merger contributes a large amount of low-metallicity gas and angular momentum, from which the kinematically cold low-$\alpha$ stellar disk is subsequently born. This simulated history parallels several observed features of the Milky Way, particularly the decisive 'GSE' merger that likely occurred at $z \approx 2$. Our results provide an all-sky perspective on the emerging picture of our Galaxy's three-phase formation, impelled by the three physical mechanisms of spinup, merger, and cooldown. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 37</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

304  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
